In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))

In [2]:
trip = validation_data_input.get_data('trip')

In [3]:
# Try to load PSRC database if available from Elmer
try:
    trip_elmer = validation_data_input.load_elmer_table("HHSurvey.v_trips_labels", 
                                              sql="SELECT * FROM HHSurvey.v_trips_labels"+\
                                                  " WHERE survey_year in ("+input_config['base_year']+")")
except:
    trip_elmer = pd.DataFrame()

Total Tours

In [5]:
df = trip.groupby('source').sum()[['trexpfac']].reset_index()
df.rename(columns={'trexpfac': 'Total Trips'}, inplace=True)

# If Elmer Data is available, use it instead of the Daysim-formatted survey
if not trip_elmer.empty:
    total_trips_elmer = trip_elmer['trip_weight'].sum()
    df = df.append({'source': 'Full Survey Data', 'Total Trips': total_trips_elmer}, ignore_index=True)
df['Total Trips'] = df['Total Trips'].apply(lambda x: f"{x:,.0f}")
df

,source,Total Trips
0,model,"16,444,886"
1,survey,"13,902,503"
2,survey (update),"14,231,730"
3,Full Survey Data,"16,125,820"
